### Contextual Chunk Headers (CCH)

#### Overview

###### Contextual chunk headers (CCH) is a method of creating chunk headers that contain higher-level context (such as document-level or section-level context), and prepending those chunk headers to the chunks prior to embedding them. This gives the embeddings a much more accurate and complete representation of the content and meaning of the text. In our testing, this feature leads to a substantial improvement in retrieval quality. In addition to increasing the rate at which the correct information is retrieved, CCH also reduces the rate at which irrelevant results show up in the search results. This reduces the rate at which the LLM misinterprets a piece of text in downstream chat and generation applications.

#### Motivation
###### Many of the problems developers face with RAG come down to this: Individual chunks oftentimes do not contain sufficient context to be properly used by the retrieval system or the LLM. This leads to the inability to answer questions and, more worryingly, hallucinations.

#### Examples of this problem

###### Chunks oftentimes refer to their subject via implicit references and pronouns. This causes them to not be retrieved when they should be, or to not be properly understood by the LLM.
###### Individual chunks oftentimes only make sense in the context of the entire section or document, and can be misleading when read on their own.

#### Key Components
#### Contextual chunk headers
###### The idea here is to add in higher-level context to the chunk by prepending a chunk header. This chunk header could be as simple as just the document title, or it could use a combination of document title, a concise document summary, and the full hierarchy of section and sub-section titles.

#### Method Details
###### Context generation
###### In the demonstration below we use an LLM to generate a descriptive title for the document. This is done through a simple prompt where you pass in a truncated version of the document text and ask the LLM to generate a descriptive title for the document. If you already have sufficiently descriptive document titles then you can directly use those instead. We've found that a document title is the simplest and most important kind of higher-level context to include in the chunk header.

###### Other kinds of context you can include in the chunk header:

###### Concise document summary
###### Section/sub-section title(s)
###### This helps the retrieval system handle queries for larger sections or topics in documents.
###### Embed chunks with chunk headers
###### The text you embed for each chunk is simply the concatenation of the chunk header and the chunk text. If you use a reranker during retrieval, you'll want to make sure you use this same concatenation there too.

###### Add chunk headers to search results
###### Including the chunk headers when presenting the search results to the LLM is also beneficial as it gives the LLM more context, and makes it less likely that it misunderstands the meaning of a chunk.

#### Setup
###### You'll need a Cohere API key and an OpenAI API key for this notebook.

In [ ]:
import cohere
import tiktoken
from typing import List
from openai import OpenAI
import os
from dotenv import load_dotenv
from langchain_text_splitters import RecursiveCharacterTextSplitter

#Load environment variables from a .env file
load_dotenv()
os.environ["CO_API_KEY"] = os.getenv("CO_API_KEY") #Cohere API key
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY') #OpenAI API key


### Load the document and split it into chunks
###### We'll use the basic LangChain RecursiveCharacterTextSplitter for this demo, but you can combine CCH with more sophisticated chunking methods for even better performance.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split